In [ ]:
!pip install flash_attn
!pip install langchain
!pip install -qU langchain-community faiss-cpu
!pip install -qU langchain-openai
!pip install --upgrade --quiet  rank_bm25
!pip install langchain_experimental
!pip install -U FlagEmbedding
!pip install -qq scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash_attn
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━

In [ ]:
import pandas as pd
data = pd.read_csv("query_docs.csv")

In [ ]:
data

,Query,Docs (Index)
0,Trà hoa cúc,"3, 36, 65, 70"
1,Trà hỗ trợ ung thư,"0, 7, 12, 17, 21, 23, 26, 31, 36, 46, 47, 54, ..."
2,Sản phẩm giúp ăn ngon miệng,"0, 3, 5, 9, 10, 12, 18, 31, 55, 64, 72, 85, 99..."
3,Cao atiso,"5, 10, 18, 32, 33, 101, 102, 103"
4,Trà thượng hạng,"6, 20, 59, 88, 104, 105"
...,...,...
56,Có công dụng giải rượu,"8, 28, 60, 86"
57,Sản phẩm giúp đen tóc,"16, 27, 97"
58,Có tác dụng giảm đau bụng kinh,"2, 4, 19, 49, 51, 65, 71, 73, 74, 76, 99"
59,"Giúp thông kinh mạch, tốt cho người già",22


In [ ]:
import json
from typing import List
from langchain_core.documents import Document

def load_chunks_from_file(input_file: str = "chunks_data.json") -> List[Document]:
    """Loads chunks from a JSON file and returns them as a list of Documents."""
    with open(input_file, "r", encoding="utf-8") as f:
        chunk_data = json.load(f)

    list_of_chunks = []
    for chunk_info in chunk_data:
        chunk = Document(
            page_content=chunk_info["content"],
            metadata=chunk_info["metadata"],
        )
        list_of_chunks.append(chunk)

    print(f"Loaded {len(list_of_chunks)} chunks from {input_file}")
    return list_of_chunks

In [ ]:
list_of_chunks = load_chunks_from_file('/content/drive/MyDrive/Information Retrieval/chunks_uitnlp--visobert.json')

Loaded 332 chunks from /content/drive/MyDrive/Information Retrieval/chunks_uitnlp--visobert.json


In [ ]:
from typing import List, Callable
import faiss
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import pickle
EmbeddingFunction = Callable[[List[str]], List[List[float]]]
import os

def load_vector_store(
    embedding_function: EmbeddingFunction,
    model_name: str,
    output_dir: str = "data/vector_store",
) -> FAISS:
    """Loads a FAISS vector store from files."""
    save_model_name = model_name.replace("/", "--").replace(" ", "_")
    index_file = os.path.join(output_dir, f"faiss_index_{save_model_name}.bin")
    metadata_file = os.path.join(output_dir, f"metadata_{save_model_name}.pkl")

    index = faiss.read_index(index_file)
    with open(metadata_file, "rb") as f:
        metadata = pickle.load(f)
    docstore = metadata["docstore"]
    index_to_docstore_id = metadata["index_to_docstore_id"]

    vector_store = FAISS(
        embedding_function=embedding_function,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
    )

    print(f"FAISS index loaded from {index_file} and metadata from {metadata_file}")
    return vector_store

In [ ]:
from google.colab import userdata
import os

embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=1024)

In [ ]:
from sentence_transformers import SentenceTransformer


sentence_transformer = SentenceTransformer("hiieu/halong_embedding")
embedding_dimension = sentence_transformer.get_sentence_embedding_dimension()

# Wrap the sentence transformer encode for batched input
def batch_encode(texts: List[str]) -> List[List[float]]:
    return sentence_transformer.encode(texts).tolist()

embedding_func = batch_encode

In [ ]:
vector_store = load_vector_store(embedding_func, "hiieu/halong_embedding", '/content/drive/MyDrive/Information Retrieval' )

FAISS index loaded from /content/drive/MyDrive/Information Retrieval/faiss_index_hiieu--halong_embedding.bin and metadata from /content/drive/MyDrive/Information Retrieval/metadata_hiieu--halong_embedding.pkl


In [ ]:
semantic_results = vector_store.similarity_search(
    "Trà hoa cúc",
    k=10,
)
for res in semantic_results:
    print(f"* {res.page_content} [{res.metadata}]")

* Mỗi khi nóng trong người lại lấy trà hoa cúc ra nấu như một dạng nước mát. [{'parent': 4}]
* Tên sản phẩm: TRÀ TÚI LỌC CUNG ĐÌNH HUẾ - Giá gốc: 60,000đ - Giá khuyến mãi: 60,000đ - Danh mục sản phẩm: Sản phẩm Trà - Mô tả: Thành phần:16 vị thảo mộc cung đình Huế.Nguồn gốc:Trà Đức Phượng - HuếQuy cách đóng gói:1 hộp gồm 30 túi lọc trà nhỏ.Bảo quản:nhiệt độ phòng, đậy kín sau khi sử dụng.Cam kết:không dùng hoá chất bảo quản.Trà cung đình Huế dạng túi lọc, tiện dụng, hợp với người bận rộn, làm việc văn phòng,... thành phần vẫn đầy đủ gồm 16 vị thảo mộc, xay nhỏ và đóng thành trà túi lọc nên chất lượng đảm bảo không đổi. Trung bình một hộp dùng được 10-15 ngày nên tính ra khá rẻ. - Chi tiết: Trà cung đình Huế túi lọc-, tiện dụng, hợp với người bận rộn, làm việc văn phòng,... [{'parent': 82}]
* Mọi thông tin đặt hàng vui lòng liên hệ với các kênh bán hàng của shop nhé. - Chi tiết: Trà quý phi cung đình huế-- trà dành cho phái nữ, uống hàng ngày giúp giảm cân, đẹp da,... với thành phần từ 17

In [ ]:
# Using BM25 retriever from LangChain
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(
  list_of_chunks, k = 10
)

bm25_results = bm25_retriever.invoke("Trà hoa cúc")
bm25_results

content = set()
retrieval_docs = []

for result in semantic_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

for result in bm25_results:
  if result.page_content not in content:
    content.add(result.page_content)
    retrieval_docs.append(result)

len(retrieval_docs)

17

In [ ]:
!pip install flash_attn py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 37.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=3a0e0b8154acb5d8a49489656a87204261594edb4874a5ec7050e62b1ceafea1
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/usr/local/lib/python3.10/dist-packages/py_vncorenlp')
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/usr/local/lib/python3.10/dist-packages/py_vncorenlp')

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # S
pairs = [["Trà hoa cúc",doc.page_content] for doc in retrieval_docs]
score = reranker.compute_score(pairs,normalize = True)



tokenized_query = rdrsegmenter.word_segment('Trà hoa cúc')
tokenized_sentences  = [rdrsegmenter.word_segment(doc.page_content) for doc in retrieval_docs]
joined_query = " ".join(tokenized_query)
joined_sentences = [" ".join(sent) for sent in tokenized_sentences]
tokenized_pairs = [[joined_query, sent] for sent in joined_sentences]
MODEL_ID = 'itdainb/PhoRanker'
MAX_LENGTH = 256

# from sentence_transformers import CrossEncoder
# model = CrossEncoder(MODEL_ID, max_length=MAX_LENGTH)

# # For fp16 usage
# model.model.half()

# scores = model.predict(tokenized_pairs)

In [ ]:
class Retriever:
    def __init__(self, semantic_retriever, bm25_retriever, reranker, k_docs):
        self.semantic_retriever = semantic_retriever
        self.bm25_retriever = bm25_retriever
        self.reranker = reranker
        self.k_docs = k_docs

    def __call__(self, query, mode="hybrid"):
        """
        Retrieves relevant documents based on the specified mode.

        Args:
            query (str): The search query.
            mode (str): The retrieval mode to use. Options are:
                - "bm25": Only uses BM25 retrieval.
                - "semantic": Only uses semantic retrieval.
                - "hybrid" (default): Combines both semantic and BM25 retrieval.

        Returns:
            list: A list of parent IDs of the top k_docs retrieved documents, sorted by reranker score.
        """
        retrieval_docs = []
        content = set()

        if mode == "bm25":
            bm25_results = self.bm25_retriever.invoke(query)
            for result in bm25_results:
                if result.page_content not in content:
                    content.add(result.page_content)
                    retrieval_docs.append(result)
        elif mode == "semantic":
            semantic_results = self.semantic_retriever.similarity_search(
                query,
                k=10,
            )
            for result in semantic_results:
                if result.page_content not in content:
                    content.add(result.page_content)
                    retrieval_docs.append(result)
        elif mode == "hybrid":
            semantic_results = self.semantic_retriever.similarity_search(
                query,
                k=10,
            )
            for result in semantic_results:
                if result.page_content not in content:
                    content.add(result.page_content)
                    retrieval_docs.append(result)

            bm25_results = self.bm25_retriever.invoke(query)
            for result in bm25_results:
                if result.page_content not in content:
                    content.add(result.page_content)
                    retrieval_docs.append(result)
        else:
            raise ValueError(f"Invalid retrieval mode: {mode}. Available modes are 'bm25', 'semantic', and 'hybrid'.")

        if not retrieval_docs:
            return []

        pairs = [[query, doc.page_content] for doc in retrieval_docs]
        tokenized_query = rdrsegmenter.word_segment(query)
        tokenized_sentences  = [rdrsegmenter.word_segment(doc.page_content) for doc in retrieval_docs]
        joined_query = " ".join(tokenized_query)
        joined_sentences = [" ".join(sent) for sent in tokenized_sentences]
        tokenized_pairs = [[joined_query, sent] for sent in joined_sentences]

        self.reranker.model.half()
        scores = self.reranker.predict(tokenized_pairs)

        # scores = self.reranker.compute_score(pairs, normalize=True)
        parent_ids_with_scores = []
        for i in range(len(retrieval_docs)):
            parent_idx = retrieval_docs[i].metadata['parent']
            parent_ids_with_scores.append((parent_idx, scores[i]))

        sorted_idx = sorted(parent_ids_with_scores, key=lambda item: item[1], reverse=True)
        sorted_idx = [item[0] for item in sorted_idx]
        return sorted_idx[:self.k_docs]

In [ ]:
retriever = Retriever(semantic_retriever = vector_store, bm25_retriever = bm25_retriever, reranker = model, k_docs=10)


In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

def evaluate_query(query, ground_truth_indices, retriever, k=10, mode="hybrid"):
    """Evaluates a query for a given retriever and mode."""
    retrieved_indices = retriever(query, mode=mode)
    retrieved_indices_k = retrieved_indices[:k]
    retrieved_set = set(retrieved_indices_k)
    relevant_set = set(ground_truth_indices)

    true_positives = len(retrieved_set.intersection(relevant_set))
    false_positives = len(retrieved_set - relevant_set)
    false_negatives = len(relevant_set - retrieved_set)

    if true_positives + false_positives > 0:
        precision = true_positives / (true_positives + false_positives)
    else:
        precision = 0.0

    if true_positives + false_negatives > 0:
        recall = true_positives / (true_positives + false_negatives)
    else:
        recall = 0.0

    if precision + recall > 0:
        f1 = 2 * (precision * recall) / (precision + recall)
    else:
        f1 = 0.0

    # MRR Calculation
    rr = 0
    for i, retrieved_index in enumerate(retrieved_indices_k):
        if retrieved_index in relevant_set:
            rr = 1 / (i + 1)
            break

    # Hit Rate Calculation
    hit_rate = 1 if true_positives > 0 else 0

    # NDCG Calculation
    dcg = 0
    for i, retrieved_index in enumerate(retrieved_indices_k):
        if retrieved_index in relevant_set:
            dcg += 1 / np.log2(i + 2)

    idcg = 0
    for i in range(min(k, len(relevant_set))):
        idcg += 1 / np.log2(i + 2)

    ndcg = dcg / idcg if idcg > 0 else 0

    return precision, recall, f1, rr, hit_rate, ndcg

In [ ]:

modes = ["bm25", "semantic", "hybrid"]
all_metrics = {}

k = 10
data = pd.read_csv("/content/drive/MyDrive/Information Retrieval/query_docs.csv")

for mode in modes:
    precisions = []
    recalls = []
    f1s = []
    mrrs = []
    hit_rates = []
    ndcgs = []

    print(f"Evaluating with mode: {mode}")
    for index, row in tqdm(data.iterrows(), total=len(data), desc=f"Evaluating Queries ({mode})"):
        query = row['Query']
        ground_truth_str = row['Docs (Index)']

        # Handle cases where there's only one index (not a comma-separated string)
        if isinstance(ground_truth_str, str) and ',' in ground_truth_str:
            ground_truth_indices = [int(x.strip()) for x in ground_truth_str.split(',')]
        else:
            try:
                ground_truth_indices = [int(ground_truth_str)]
            except ValueError:
                print(f"Skipping row {index} due to invalid index format: {ground_truth_str}")
                continue

        precision, recall, f1, rr, hit_rate, ndcg = evaluate_query(
            query, ground_truth_indices, retriever, k=k, mode=mode
        )
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        mrrs.append(rr)
        hit_rates.append(hit_rate)
        ndcgs.append(ndcg)

    # Calculate the average metrics for the current mode
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1 = np.mean(f1s)
    avg_mrr = np.mean(mrrs)
    avg_hit_rate = np.mean(hit_rates)
    avg_ndcg = np.mean(ndcgs)

    all_metrics[mode] = {
        "avg_precision": avg_precision,
        "avg_recall": avg_recall,
        "avg_f1": avg_f1,
        "avg_mrr": avg_mrr,
        "avg_hit_rate": avg_hit_rate,
        "avg_ndcg": avg_ndcg,
    }

# Print the results for each mode
print("\nEvaluation Results per Mode:")
for mode, metrics in all_metrics.items():
    print(f"Mode: {mode}")
    print(f"  Average Precision@{k}: {metrics['avg_precision']:.4f}")
    print(f"  Average Recall@{k}: {metrics['avg_recall']:.4f}")
    print(f"  Average F1-Score@{k}: {metrics['avg_f1']:.4f}")
    print(f"  Average MRR@{k}: {metrics['avg_mrr']:.4f}")
    print(f"  Hit Rate@{k}: {metrics['avg_hit_rate']:.4f}")
    print(f"  Average NDCG@{k}: {metrics['avg_ndcg']:.4f}")
    print("-" * 30)

Evaluating with mode: bm25


Evaluating Queries (bm25):   0%|          | 0/61 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Evaluating with mode: semantic


Evaluating Queries (semantic):   0%|          | 0/61 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Evaluating with mode: hybrid


Evaluating Queries (hybrid):   0%|          | 0/61 [00:00<?, ?it/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai


Evaluation Results per Mode:
Mode: bm25
  Average Precision@10: 0.4391
  Average Recall@10: 0.6195
  Average F1-Score@10: 0.4466
  Average MRR@10: 0.8267
  Hit Rate@10: 0.9672
  Average NDCG@10: 0.8415
------------------------------
Mode: semantic
  Average Precision@10: 0.2815
  Average Recall@10: 0.3582
  Average F1-Score@10: 0.2734
  Average MRR@10: 0.7124
  Hit Rate@10: 0.8361
  Average NDCG@10: 0.6023
------------------------------
Mode: hybrid
  Average Precision@10: 0.4039
  Average Recall@10: 0.5498
  Average F1-Score@10: 0.3923
  Average MRR@10: 0.8390
  Hit Rate@10: 0.9672
  Average NDCG@10: 0.8698
------------------------------
